# KarriereAI
#### A deep learning model used to predict viable career paths to a user based on their skills and interests.

#### Purpose
KarriereAI will classify the appropriate career within technology for a user from an interactive quizlet input. The quizlet is used to determine the user's skills and interests, before the model will predict a fitting career based on the input data.

#### Dataset
The dataset to be used is from the paper Skill2vec: A Machine Learning Approach for Determining the Relevant Skills from Job Description, by Van-Duyet Le et al. <a href="https://arxiv.org/pdf/1707.09751">here</a>. Containing relevant columns of job titles related to a free-text field of job descriptions describing relevant skills. 

#### Model Architecture
The main model is a feed-forward neural network (FNN) for classification, using a multi-layer-perceptron (MLP) architecture suitable for structured data classification. 

Part of engineering the main model requires preprocessing of the dataset in a natural language processing (NLP) set-up, preparing it to run through a sub-model with an encoded transformer architecture.

The main model also requires figuring out basic vs. deep MLP architecures. Additionally, figuring out whether the model should contain batch normalization, regularization and/or dropout. Lastly, experimenting with different activation functions.

Through using the NLP sub-model, the data will be vectorized to work with the MLP classifier rather than the language model.

#### Evaluation
As far as evaluation goes, a confusion matrix and an F1 score will be computed along with standard evaluation metrics like accuracy, recall and precision.

### Step 1 - Importing Libraries and Loading the Data 
We will be needing different libraries from <a href="https://keras.io/api/">Keras</a> and <a href="https://www.tensorflow.org/api_docs/python/tf">TensorFlow</a> to make computations on the dataset.

In [26]:
# Model processing
import sklearn
import numpy
import pandas 
import tensorflow
import keras

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras import layers, Sequential
from scipy import sparse
from scipy.sparse import hstack

# Other
import datetime

# For plotting
%matplotlib inline
import matplotlib as plot
import matplotlib.pyplot as pyplot

from pathlib import Path
import seaborn
seaborn.set_theme (style = "whitegrid")

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


After importing relevant libraries, we load the dataset we wish to train the model on.

In [ ]:
# Load dataset
data = pandas.read_csv ("data/mustHaveSkills.csv", header = 0, encoding ='ISO-8859-1')
del data ['job_brief_id']

# REF: Van-Duyet Le

### Step 2 - Taking a Look at the Data

To create optimal and smooth-running Python for the model we want to  study the shape of the data.

In [ ]:
# Basic information about dataset
print ("Shape of dataset:", data.shape, "\n")
print ("Information about dataset:")
data.info()

Shape of data: (261724, 2) 

Information about data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261724 entries, 0 to 261723
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   keyword_name  261717 non-null  object
 1   job_title     261724 non-null  object
dtypes: object(2)
memory usage: 4.0+ MB


In [ ]:
# Drop duplicates
data = data.drop_duplicates (subset = ['keyword_name', 'job_title'], keep = 'last')
data = data [data ["job_title"] != 0]
print (data.info())

# String magic
data ['Count'] = data.groupby ('job_title')['keyword_name'].transform (pandas.Series.value_counts)
data.drop_duplicates (inplace = True)
data ['keyword_name'] = data ['keyword_name'].str.lower()
data ['keyword_name'] = data ['keyword_name'].str.replace(' ', '_')
data ['job_title'] = data ['job_title'].str.lower()

# Clean data
data_jobtitle = data.groupby ('job_title')['keyword_name'].apply(list)

# REF: Van-Duyet Le

<class 'pandas.core.frame.DataFrame'>
Index: 80810 entries, 0 to 261723
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   keyword_name  80803 non-null  object
 1   job_title     80810 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB
None


In [ ]:
# Look at contents of dataset
data_jobtitle

job_title
.net                             [microsoft_office_sharepoint_server]
.net application                       [architect, .net, asp.net, c#]
.net developer      [.net, sharepoint, moss, asp.net, html/html5, ...
.net engineer       [.net, .net, asp.net, javascript, mobile_appli...
.net programmer                                                [.net]
                                          ...                        
xbox security                            [c++, xbox, ps, playstation]
xd                  [ux_design, ux, uxd, ued, xd, user_experience_...
xml developer                                        [xml, html, css]
yield manager       [yield, online_media, digital, advertising, br...
zbb analysts           [zerbased_budgeting, zbb, accounting, finance]
Name: keyword_name, Length: 4596, dtype: object

### Step 3 - Exploratory Data Analysis (EDA)

To know what data the model will injest, taking a closer look by constructing plots and 

#### 3.1 Plotting Jobs and Skills
##### 3.1.1 Jobs

In [ ]:
# Function to count number of occurrences 
def count_items(series):
    items = series.dropna().apply(lambda x: x.split(";"))
    flat_list = [item.strip() for sublist in items for item in sublist]
    return pandas.Series(flat_list).value_counts()

# REF: Adil Shamim

In [ ]:
# Count Jobs
job_count = count_items(data["job_title"])
print("Most Common Jobs:\n", job_count)

Most Common Jobs:
 software engineer                      1583
software developer                      941
engineer                                891
manager                                 853
developer                               663
                                       ... 
js engineer                               1
js programmer                             1
industrial designer                       1
third party marketing games manager       1
marketing games manager                   1
Name: count, Length: 4597, dtype: int64


##### 3.1.2 Skills

In [33]:
# Count Skills
skills_count = count_items(data["keyword_name"])
print("Most Common Skills:\n", skills_count)

Most Common Skills:
 java                          771
javascript                    500
python                        473
c++                           447
sql                           361
                             ... 
mdd                             1
clinical_evaluation_report      1
cer                             1
vaccines                        1
actuate_report                  1
Name: count, Length: 6876, dtype: int64


### Step 4: Preprocessing of Data

For the purpose of cleaning the dataset to make sure the model doesn't learn errors making predictions skewed.

#### 4.1 Missing Values
Checking for missing values and dropping them if they few enough to make less of an impact. Luckily for us, there are no missing values in this dataset.

In [34]:
# missing_values = data.isnull().sum()
# print ('Missing Values in Each Column')
# print (missing_values)

#### 4.2 Feature Definition

In [35]:
# Define features to be used
features = data [['job_title', 'keyword_name']]
target = data ['job_title']

#### 4.3 Process Text Fields
Turning skills and interests into vectorized numerical features for classification.

In [ ]:
data = []

for vector_list in data_jobtitle:
    vlist = list (set (vector_list))
    if vlist not in data:
        data.append (vlist)
        
len (data)
data [0:2]

# REF: Van-Duyet Le

In [ ]:


data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
# drop duplicate
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')
# lower-case
data_naruki['skill'] = data_naruki['skill'].str.lower()
data_naruki['skill'] = data_naruki['skill'].str.replace(' ','_')
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print (len (data_naruki_final))
data_naruki_final[:10]



SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (4144046653.py, line 8)

In [ ]:
# Creating a combined text column
features['Text_Features'] = features ['Skills'].fillna('') + ' ' + features ['Interests'].fillna('')

# Text Feature Extraction
vectorizer = CountVectorizer (tokenizer = lambda x: [item.strip() for item in x.split(";") if item.strip()], lowercase = True)

text_features = vectorizer.fit_transform(features['Text_Features'])

# # Text based matrix from Skills
# skills_matrix = vectorizer.fit_transform (features ['Skills'])

# # Text based matrix from Interests
# interests_matrix = vectorizer.fit_transform (features ['Interests'])

# # Numeric matrix from Age
# age_matrix = features['Age'].values

# skills_and_interests_matrix = vectorizer.fit_transform (features ['Skills_and_Interests'])
print ("Shape of text features:", text_features.shape)

Shape of text features: (200, 150)


C:\Users\arege\AppData\Local\Temp\ipykernel_1116\2984523955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Text_Features'] = features ['Skills'].fillna('') + ' ' + features ['Interests'].fillna('')
c:\Users\arege\Documents\Dataingenior Bachelor\DAT255\KarriereAI\venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#### 4.4 Creating a Vocabulary

In [ ]:
import tensorflow_text
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab


In [ ]:
# Using the BERT vocabulary generation method from Tensorflow Text

bert_tokenizer_params = dict (lower_case = True)
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens = reserved_tokens,
    
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params = bert_tokenizer_params,
    
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params = {},
)



In [ ]:
%%time
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch (1000).prefetch(2),
    **bert_vocab_args
)


In [ ]:
print(pt_vocab[:10])
print(pt_vocab[100:110])
print(pt_vocab[1000:1010])
print(pt_vocab[-10:])


In [ ]:
def write_vocab_file (filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file = f)


In [ ]:
write_vocab_file ('pt_vocab.txt', pt_vocab)


#### 4.5 Tokenize Vocabulary

In [ ]:
# Using the BERT tokenizer on the newly generated vocabulary


#### 4.4 Constructing the Feature Matrix

Combining the numerical value of Age with the vectorized Skills and Interest features into a sparse matrix concatenation.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Numeric matrix from Age
age_matrix = features[['Age']].values # ? Updated to double-bracket

# Combining the matrices
feature_matrix = hstack ([sparse.csr_matrix (age_matrix), text_features]) # X
coo = feature_matrix.tocoo()
indices = numpy.asmatrix([coo.row, coo.col]).transpose()
feature_matrix = tensorflow.SparseTensor(indices, coo.data, coo.shape)

# Encoding target variable
target_encoder = LabelEncoder ()
encoded_target = target_encoder.fit_transform (target) # Y

print ("Feature Matrix Shape:", feature_matrix.shape) # X
print ("Target Classes:", target_encoder.classes_) # Y

Feature Matrix Shape: (200, 151)
Target Classes: ['AI Researcher' 'AI Specialist' 'Automation Engineer' 'Backend Developer'
 'Biostatistician' 'Business Analyst' 'Cloud Engineer'
 'Content Strategist' 'Cybersecurity Analyst' 'Cybersecurity Specialist'
 'Data Analyst' 'Data Engineer' 'Data Scientist' 'Deep Learning Engineer'
 'DevOps Engineer' 'Digital Marketer' 'Embedded Systems Engineer'
 'Financial Analyst' 'Front-end Developer' 'Full Stack Developer'
 'Graphic Designer' 'Machine Learning Engineer' 'Marketing Manager'
 'Mobile Developer' 'NLP Engineer' 'Project Manager' 'Research Analyst'
 'Research Scientist' 'Software Developer' 'Software Engineer'
 'UX Designer' 'UX Researcher']


### Step 5 - Building the Model

Building the FFN model, and training it on training and validation sets.

#### 5.1 Training, Validation and Test Sets

In [ ]:
from sklearn.model_selection import train_test_split

print (type(feature_matrix), type(encoded_target))

X_train = feature_matrix 
y_train = encoded_target 

X_train = tensorflow.sparse.reorder(X_train)

# X_train, X_test, y_train, y_test = train_test_split (feature_matrix, encoded_target, test_size = 0.2, random_state = 42)

# tensorflow.sparse.reorder(X_train)

# this is trash:
# training_set, validation_set, test_set = train_val_test_split (age_matrix, feature_matrix, )
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

<class 'tensorflow.python.framework.sparse_tensor.SparseTensor'> <class 'numpy.ndarray'>


#### 5.2 Training the Model

In [ ]:
# Remove previous logs
# TODO

In [ ]:
model = Sequential([
    keras.Input (shape = (feature_matrix.shape[1], )),
    # to complete the embedding, a one-hot layer:layers.StringLookup (output_mode = "one-hot")
    layers.Dropout (0.1),
    layers.Dense (16, activation = 'relu'),
    layers.Dense (16, activation = 'relu'),
    # followed by a Dense layer: layers.Dense (units = embedding_dim, use_bias = False, activation = None)
    layers.Dense (target.unique().size)
])

# model.compile(loss = 'SparseCategoricalCrossentropy', optimizer = 'adam')
model.compile (loss = 'hinge', 
              optimizer = 'adam', 
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

model.fit (X_train, y_train, 
          epochs = 30, 
          batch_size = 1, 
          validation_split = 0.2,
          callbacks=[tensorboard_callback])

Epoch 1/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0437 - loss: 0.5685 - val_accuracy: 0.0250 - val_loss: 0.0411
Epoch 2/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0378 - loss: 0.0684 - val_accuracy: 0.0250 - val_loss: 0.0181
Epoch 3/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0841 - loss: 0.0267 - val_accuracy: 0.0250 - val_loss: 0.0108
Epoch 4/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0272 - loss: 0.0433 - val_accuracy: 0.0000e+00 - val_loss: 0.0097
Epoch 5/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0524 - loss: 0.0202 - val_accuracy: 0.0250 - val_loss: 0.0147
Epoch 6/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0425 - loss: 0.0249 - val_accuracy: 0.0000e+00 - val_loss: 0.0098
Epoch 7/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0897 - loss: 0.0266 - val_accuracy: 0.0000e+00 - val_loss: 0.0153
Epoch 8/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0141 - loss: 0.0606 - val_

In [ ]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_16 (Dropout)            │ (None, 151)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 16)             │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 32)             │           544 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,378 (44.45 KB)

 Trainable params: 3,792 (14.81 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,586 (29.64 KB)

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 20476), started 1 day, 13:53:40 ago. (Use '!kill 20476' to kill it.)

### Step 6 - Evaluation

#### 6.1 Confusion Matrix

In [ ]:
# Predict on the test set
# y_pred = model.predict(X_train)

In [ ]:
# # Confusion matrix
# cm = confusion_matrix(y_train, y_pred)
# plot.figure(figsize = (10, 8))
# seaborn.heatmap(cm, annot = True, fmt = "d", cmap = "Blues",
#             xticklabels = target_encoder.classes_,
#             yticklabels = target_encoder.classes_)
# plot.xlabel("Predicted")
# plot.ylabel("Actual")
# plot.title("Confusion Matrix")
# plot.show()

#### 6.2 F1-Score

#### 6.3 Precision, Accuracy, Recall